In [ ]:
import plotly.graph_objects as go
import numpy as np

GPUs = [('H100', 256_000, 50_000_000), ('A100', 192_000, 40_000_000), ('V100', 128_000, 6_000_000), ('P100', 24_000, 4_000_000)]

delays_us = range(1, 1000000)
delays_ms = [x / 1000.0 for x in delays_us]
delays_s = [x / 1000.0 for x in delays_ms]

pcie5x16 = 128
SXM5 = 900

throughputs_l1 = []
throughputs_l2 = []
intersects_l2_pci = []
intersects_l2_sxm = []
intersects_l1_pci = []
intersects_l1_sxm = []

for name, l1, l2 in GPUs:
    throughput_l2 = []
    throughput_l1 = []
    for delay in delays_s:
        t_l2 = l2 / (delay * 1_000_000)
        t_l1 = l1 / (delay * 1_000_000)
        throughput_l2.append(t_l2)
        throughput_l1.append(t_l1)

    index = np.argmin(np.abs(np.array(throughput_l2) - pcie5x16))
    intersects_l2_pci.append((index / 1_000_000, throughput_l2[index]))

    index = np.argmin(np.abs(np.array(throughput_l2) - SXM5))
    intersects_l2_sxm.append((index / 1_000_000, throughput_l2[index]))

    index = np.argmin(np.abs(np.array(throughput_l1) - pcie5x16))
    intersects_l1_pci.append((index / 1_000_000, throughput_l1[index]))

    index = np.argmin(np.abs(np.array(throughput_l1) - SXM5))
    intersects_l1_sxm.append((index / 1_000_000, throughput_l1[index]))

    throughputs_l2.append((name, throughput_l2))
    throughputs_l1.append((name, throughput_l1))

fig_l1 = go.Figure()
fig_l2 = go.Figure()

for name, throughput in throughputs_l2:
    fig_l2.add_trace(go.Scatter(x=delays_s, y=throughput, mode='lines', name=name))

for name, throughput in throughputs_l1:
    fig_l1.add_trace(go.Scatter(x=delays_s, y=throughput, mode='lines', name=name))

fig_l2.add_shape(
    type='line',
    x0=min(delays_s),
    y0=SXM5,
    x1=max(delays_s),
    y1=SXM5,
    line=dict(color='grey', width=1, dash='dot'),
    name='SXM5'
)

fig_l2.add_shape(
    type='line',
    x0=min(delays_s),
    y0=pcie5x16,
    x1=max(delays_s),
    y1=pcie5x16,
    line=dict(color='grey', width=1, dash='dash'),
    name='PCIe 5.0 x16'
)

fig_l1.add_shape(
    type='line',
    x0=min(delays_s),
    y0=SXM5,
    x1=max(delays_s),
    y1=SXM5,
    line=dict(color='grey', width=1, dash='dot'),
    name='SXM5'
)

fig_l1.add_shape(
    type='line',
    x0=min(delays_s),
    y0=pcie5x16,
    x1=max(delays_s),
    y1=pcie5x16,
    line=dict(color='grey', width=1, dash='dash'),
    name='PCIe 5.0 x16'
)

scatter_l2 = []
scatter_l1 = []

for delay, throughput in intersects_l2_sxm:
    scatter_l2.append(go.Scatter(x=[delay], y=[throughput], mode='markers', name=f'({round(delay, 6)}, {round(throughput, 0)})'))

for delay, throughput in intersects_l2_pci:
    scatter_l2.append(go.Scatter(x=[delay], y=[throughput], mode='markers', name=f'({round(delay, 6)}, {round(throughput, 0)})'))

for delay, throughput in intersects_l1_sxm:
    scatter_l1.append(go.Scatter(x=[delay], y=[throughput], mode='markers', name=f'({round(delay, 6)}, {round(throughput, 0)})'))

for delay, throughput in intersects_l1_pci:
    scatter_l1.append(go.Scatter(x=[delay], y=[throughput], mode='markers', name=f'({round(delay, 6)}, {round(throughput, 0)})'))

fig_l2.add_traces(scatter_l2)
fig_l1.add_traces(scatter_l1)

fig_l2.update_xaxes(type='log', title='Delay (s)', exponentformat='power')
fig_l2.update_yaxes(type='log', title='Throughput (GB/s)', exponentformat='power')
fig_l2.update_layout(title='Throughput vs Delay based on L2 cache size',     autosize=False,
    width=700,
    height=500,)

fig_l1.update_xaxes(type='log', title='Delay (s)', exponentformat='power')
fig_l1.update_yaxes(type='log', title='Throughput per SM (GB/s)', exponentformat='power')
fig_l1.update_layout(title='Throughput per SM vs Delay based on L1 cache size',     autosize=False,
    width=700,
    height=500,)

fig_l1.show()
fig_l2.show()
